In [25]:
import numpy as np
from typing import Callable, Union, Tuple, List, Any

# Initial-Value Problems for Ordinary Differential Equations

## Chapter 5.2 Euler's Method
Given 
$$ \frac{dy}{dt} = f(t, y),\,\,\, a\leq t\leq b,\,\,\,y(a)=\alpha $$
and consider
$$ t_i = a+ih,\,\,\,i=0,1,2\cdots,N$$
Suppose $y(t)$ has two continuous derivatives on $[a, b]$, then by Taylor's Theorem
$$ y(t_{i+1})=y(t_i)+(t_{i+1}-t_i)y'(t_i)+\frac{(t_{i+1}-t_i)^2}{2}y''(\xi_i) $$
because $h=t_{i+1}-t_i$, then
$$ y(t_{i+1})=y(t_i)+hf(t_i,y(t_i))+\frac{h^2}{2}y''(\xi_i)$$

The definition of $t_i$ and $h$ are the same throughout the context.

Euler's method constructs $w_i\approx y(t_i)$, by deleting the remainder term. In short,
$$w_0=\alpha$$
$$w_{i+1}=w_i+hf(t_i, w_i),\,\,\,i=0, 1,\cdots,N-1$$

In [26]:
def Euler(f:Callable[[Union[float, int], Union[float, int]], Union[float, int]],
          a:Union[float, int],
          b:Union[float, int],
          alpha:Union[float, int],
          h:Union[float, int]=None,
          N:int=None,
         ) -> Tuple[np.ndarray[Union[float, int], Any], np.ndarray[Union[float, int], Any]]:
    if (h is None) and (N is None):
        raise ValueError("h and N cannot be None at the same time")
    elif h is None:
        h = (b - a) / N
    elif N is None:
        N = int((b - a) / h)
    t_, w_ = np.linspace(a, b, N + 1), np.ones((N + 1, )) * alpha
    for idx in range(1, N + 1):
        w_[idx] =  w_[idx - 1] + h * f(t_[idx - 1], w_[idx - 1])
    return (t_, w_)

In [27]:
f = lambda t, y: y - t ** 2 + 1
Euler(f, 0, 2, 0.5, h=0.5)

(array([0. , 0.5, 1. , 1.5, 2. ]),
 array([0.5   , 1.25  , 2.25  , 3.375 , 4.4375]))

In [28]:
f = lambda t, y: np.cos(2 * t) + np.sin(3 * t)
Euler(f, 0, 1, 1, N=4)

(array([0.  , 0.25, 0.5 , 0.75, 1.  ]),
 array([1.        , 1.25      , 1.63980533, 2.02425465, 2.23645725]))

## Chapter 5.3 Higher-Order Taylor Methods
Suppose the solution $y(t)$ to the initial-value problem
$$ \frac{dy}{dt} = f(t, y),\,\,\, a\leq t\leq b,\,\,\,y(a)=\alpha $$
has $(n+1)$ continuous derivatives. By its $n$th Taylor polynomial about $t_i$ and evaluate at $t_{i+1}$,
$$y(t_{{i+1}})=y(t_i)+hy'(t_i)+\frac{h^2}{2}y''(t_i)+\cdots+\frac{h^n}{n!}y^{(n)}(t_i)+\frac{h^{n+1}}{(n+1)!}y^{(n+1)}(\xi_i)$$
for some $\xi_i\in(t_i, t_{i+1})$
Successive differentiation of the solution, $y(t)$ gives
$$y'(t)=f(t, y(t)),\,\,\,y''(t)=f'(t, y(t)),\,\,\,\text{ and generally, }\,\,\, y^{(k)}(t)=f^{(k-1)}(t, y(t))$$
Substituting these findings into previous equations gives
$$y(t_{{i+1}})=y(t_i)+hf(t_i)+\frac{h^2}{2}f'(t_i)+\cdots+\frac{h^n}{n!}f^{(n - 1)}(t_i)+\frac{h^{n+1}}{(n+1)!}f^{(n)}(\xi_i)$$
Similar to Euler's method, in short
$$w_0=\alpha$$
$$w_{i+1}=w_i+hT^{(n)}(t_i, w_i),\,\,\,i=0,1,\cdots,N-1$$
where
$$T^{(n)}(t_i, w_i)=\sum^{n-1}_{i=0}\frac{h^{i}}{(i+1)!}f^{(i)}(t_i, w_i)
Euler's method is Taylor's method of order one.

In [32]:
def Taylor(func:List[Callable[[Union[float, int], Union[float, int]], Union[float, int]]],
           a:Union[float, int],
           b:Union[float, int],
           alpha:Union[float, int],
           h:Union[float, int]=None,
           N:int=None
          ) -> Tuple[np.ndarray[Union[float, int], Any], np.ndarray[Union[float, int], Any]]:
    if (h is None) and (N is None):
        raise ValueError("h and N cannot be None at the same time")
    elif h is None:
        h = (b - a) / N
    elif N is None:
        N = int((b - a) / h)
    t_, w_ = np.linspace(a, b, N + 1), np.ones((N + 1, )) * alpha

    factorial = [1] * (len(func) + 1)
    for idx in range(2, len(func) + 1):
        factorial[idx] = factorial[idx - 1] * (idx)
    for idx in range(1, N + 1):
        T = sum([(h ** jdx) * f(t_[idx - 1], w_[idx - 1]) / factorial[jdx + 1] for jdx, f in enumerate(func)])
        w_[idx] = w_[idx - 1] + h * T
    return (t_, w_)

In [33]:
f = [
    lambda t, y: y - t ** 2 + 1,
    lambda t, y: y - t ** 2 + 1 - 2 * t
]
Taylor(f, 0, 2, 0.5, 0.2)

(array([0. , 0.2, 0.4, 0.6, 0.8, 1. , 1.2, 1.4, 1.6, 1.8, 2. ]),
 array([0.5       , 0.83      , 1.2158    , 1.652076  , 2.13233272,
        2.64864592, 3.19134802, 3.74864458, 4.30614639, 4.8462986 ,
        5.34768429]))